In [1]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

Function: Load the models

In [2]:
######################################## Load the models ####################################################


#load NIST 2-BTP model, with all the halogens 
#load Nora's 2-BTP model, with all the halogens 

# Load Nora's model (145 species)

full_path_Nora = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/Nora/2_BTP/FFCM_seed/2_BTP_seed/chemkin/'

Nora_chemkin_path = full_path_Nora + 'copies/copy_chem0145.inp'
Nora_dictionary_path = full_path_Nora + 'species_dictionary.txt'
Nora_transport_path = full_path_Nora + 'tran.dat'
Nora_cti_path = full_path_Nora + 'copies/copy_chem0145.cti'

Nora_species_list, Nora_reaction_list = rmgpy.chemkin.load_chemkin_file(Nora_chemkin_path, dictionary_path=Nora_dictionary_path, transport_path=Nora_transport_path)
Nora_gas = ct.Solution(Nora_cti_path)



# Load David's model 
full_path_D = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/RMG_David/'

D_cti_path = full_path_D + 'chem_annotated.cti'
D_chemkin_path = full_path_D + 'chem_annotated.inp'
D_dictionary_path = full_path_D + 'species_dictionary.txt'
D_transport_path =  full_path_D + 'tran.dat'

D_gas = ct.Solution(D_cti_path)
D_dict = rmgpy.chemkin.load_species_dictionary(D_dictionary_path)
D_species_list, D_reaction_list = rmgpy.chemkin.load_chemkin_file(D_chemkin_path, dictionary_path= D_dictionary_path, transport_path=D_transport_path)




# Load full NIST model

full_path_NIST = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/NIST/'


NIST_cti_path = full_path_NIST + '2-BTP_kinetics.cti'
NIST_chemkin_path = full_path_NIST + 'pym_gas_copy.inp'
NIST_dictionary_path = full_path_NIST + 'species_dictionary_2-BTP.txt'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path)



**** WARNING ****
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  8.39347
	Value computed using high-temperature polynomial: 9.1801


**** WARNING ****
For species CH2OCH, discontinuity in h/RT detected at Tmid = 500
	Value computed using low-temperature polynomial:  42.1991
	Value computed using high-temperature polynomial: 41.9615


**** WARNING ****
For species CH2OCH, discontinuity in s/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  33.7069
	Value computed using high-temperature polynomial: 33.5121


**** WARNING ****
For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  47.6524
	Value computed using high-temperature polynomial: 48.4362


**** WARNING ****
For species C4H5-2, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  52.4292
	Value computed using high-temperature polyno

Functions 

In [3]:
def _2_to_1(_2_reaction):
    '''
    Convert the 2nd model species in the reactions to 1st model species, but keep the 2nd model kinetics
    '''
    
    _1_reaction = copy.deepcopy(_2_reaction)
    reactants = []
    for reactant in _2_reaction.reactants:
        try:
            _2_species_index = _2_species_list.index(reactant)
            reactants.append(_1_species_list[_2_to_1_mapping[_2_species_index]])
        except ValueError:
            if reactant in _1_species_list:
                reactants.append(reactant)
        
    _1_reaction.reactants = reactants
    
    products = []
    for product in _2_reaction.products:
        try:
            _2_species_index = _2_species_list.index(product)
            products.append(_1_species_list[_2_to_1_mapping[_2_species_index]])
        except ValueError:
            if product in _1_species_list:
                products.append(product)
    _1_reaction.products = products
    
    return _1_reaction

In [4]:
def _1_to_2(_1_reaction):
    # takes in the _1_reaction object to convert
    _1_index = _1_reaction_list.index(_1_reaction)
    if _1_index not in _1_to_2_rxn_mapping.keys():
        # this reaction does not exist in Noras, so it will be deleted. return None
        return
    _2_index = _1_to_2_rxn_mapping[_1_index]
    _2_reaction = _2_reaction_list[_2_index]
    
    # convert the N model species in the N_reaction to D model species
    return _2_to_1(_2_reaction)

Get mapping between Nora and NIST model 

In [5]:
#define the species 
_1_reaction_list = Nora_reaction_list
_1_species_list = Nora_species_list
_2_reaction_list = NIST_reaction_list
_2_species_list = NIST_species_list

# get the mapping between RMG and NIST models
# Species Diff

common_species = []
_1_to_2_mapping = {}
_2_to_1_mapping = {}
for i, _1_sp in enumerate(_1_species_list):
    for j, _2_sp in enumerate(_2_species_list):
        if _1_sp.is_isomorphic(_2_sp):
            _1_to_2_mapping[i] = j
            _2_to_1_mapping[j] = i
            common_species.append([_1_sp, _2_sp])
            break

# Reaction Diff
common_reactions = []
_1_to_2_rxn_mapping = {}
_2_to_1_rxn_mapping = {}
for i, _1_rxn in enumerate(_1_reaction_list):
    for j, _2_rxn in enumerate(_2_reaction_list):
        if _1_rxn.is_isomorphic(_2_rxn):
            _1_to_2_rxn_mapping[i] = j
            _2_to_1_rxn_mapping[j] = i
            common_reactions.append([_1_rxn, _2_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

54 common species
317 common reactions


Flip

In [6]:
############################# only know the indices of the reactions we would like to flip for Nora's model. #####

equations_for_flipping_in_Noras = ['H(8) + C3H3(6822) <=> C#CC(5272)','H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)', 'H(8) + C3H3(6822) <=> CH3(19) + C2H(22)']
index_for_flipping = []
rxns_to_change = [] #this is the list position in Nora_reaction_list


for string in equations_for_flipping_in_Noras:
    for idx,rxn in enumerate(Nora_reaction_list):
        if str(rxn)== string:
            print(rxn)
            print(idx)
            print(rxn.index)
            rxns_to_change.append(rxn.index)
            
rxns_to_change

H(8) + C3H3(6822) <=> C#CC(5272)
1271
1285
H(8) + C3H3(6822) <=> CH2(S)(25) + C2H2(23)
1307
1321
H(8) + C3H3(6822) <=> CH3(19) + C2H(22)
1308
1322


[1285, 1321, 1322]

In [7]:
########### scramble up the reactions, so that you make some models with all 3 changed, 2 changed, etc. ##########


from itertools import combinations
sample_list= rxns_to_change
list_combinations = list()
sample_set = set(sample_list)
for n in range(len(sample_set) + 1):
    list_combinations += list(combinations(sample_set, n))


final_combos = []
for i in list_combinations: 
    list_version = []
    for j in i: 
         list_version.append(j)
    final_combos.append(list_version)

final_combos




[[],
 [1321],
 [1322],
 [1285],
 [1321, 1322],
 [1321, 1285],
 [1322, 1285],
 [1321, 1322, 1285]]

In [9]:
############################### convert the indicated reactions to use Noras kinetics ##########################################

for combo in final_combos:


    rxns_to_change = combo
    print(rxns_to_change)
    # convert the indicated reactions to use the NIST kinetics

    str_rxns_to_change = []
    for index in rxns_to_change: 
        str_rxns_to_change.append(str(index))

    file_name = '_'.join(str_rxns_to_change) 
    
    
    new_reaction_list = []
    deleted_duplicates = []
    if rxns_to_change==[]:
        handpicked_dir = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/handpicked/handpicked_models'
        chemkin_file = os.path.join(handpicked_dir, f'chem_{file_name}.inp')
        rmgpy.chemkin.save_chemkin_file(chemkin_file, _1_species_list, _1_reaction_list, verbose=True, check_for_duplicates=True)
        continue
        
#     for i in _1_reaction_list:
#         if i.index in rxns_to_change:
#             print(i)
#             new_reaction = _1_to_2(_1_reaction_list[i])
#             if new_reaction:
#                 new_reaction_list.append(new_reaction)
#             elif _1_reaction_list[i].duplicate:
#                 deleted_duplicates.append(_1_reaction_list[i])
#         else:
#             new_reaction_list.append(_1_reaction_list[i])


    for i in _1_reaction_list:
        if i.index in rxns_to_change:
            print(i.index)
            new_reaction = _1_to_2(i)
            if new_reaction:
                new_reaction_list.append(new_reaction)
            elif i.duplicate:
                deleted_duplicates.append(i)
        else:
            new_reaction_list.append(i)


    # get rid of duplicates
    for i, rxn in enumerate(new_reaction_list):
        if rxn.duplicate:
            duplicate_still_exists = False
            for j, rxn2 in enumerate(new_reaction_list):
                if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                    duplicate_still_exists = True
                    break
            if not duplicate_still_exists:
                rxn.duplicate = False

    # mark reactions that are duplicates
    for i, rxn in enumerate(new_reaction_list):
        if not rxn.duplicate:
            duplicate = False
            for j, rxn2 in enumerate(new_reaction_list):
                if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                    duplicate = True
                    break
            if duplicate:
                rxn.duplicate = True

    #save the chemkin file 

    handpicked_dir = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/handpicked/handpicked_models'
    
    
    chemkin_file = os.path.join(handpicked_dir, f'chem_{file_name}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, _1_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)


[]
[1321]
1321


[1322]
1322


[1285]
1285


[1321, 1322]
1321
1322


[1321, 1285]
1285
1321


[1322, 1285]
1285
1322


[1321, 1322, 1285]
1285
1321
1322
